<div style="border:solid silver 2px; padding: 30px">
    <h2>Определение стоимости автомобилей<a class="tocSkip"> </h2> 
    
Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

---
        
**Признаки:**
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

**Целевой признак**
* Price — цена (евро)
        
---
        
**Цель**
* Построить модель, которая позволит клиентам определять рыночную стоимость своего автомобиля. 

# Подготовка данных

## Импорт библиотек и изучение общей информации

In [1]:
import pandas as pd
import time

# библиотека sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error

# модели
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.dummy import DummyRegressor

# отключение предупреждений
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
display(data.head())
display(data.info())
display(data.isna().sum())
display(data.describe())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


### Вывод

<div style="border:solid silver 2px; padding: 30px">

* Всего в датасете 16 колонок и 354369 наблюдений 
* Первая проблема, которая бросается в глаза и которую нужно исправить, - это названия столбцов в стиле, отличном от snake_case
* Что касается типов данных, то нужно будет изменить их в следующих столбцах: **DateCrawled**, **DateCreated** и **LastSeen** с object на datetime, если решим их оставить для обучения модели
* Также наблюдаются пропуски в пяти столбцах: **VehicleType**, **Gearbox**, **Model**, **FuelType** и **Repaired**, каждый из которых стоит рассмотреть подробнее
* В данных имеются аномалии, например:
    - Год регистрации автомобиля имеет минимальное значение 1000 и максимальное 9999, поэтому нужно будет определить допустимый диапазон дат, а остальные значения удалить. 
    - В столбце **Price** минимальная цена равна 0, поэтому нужно определить допустимую минимальную стоимость. Стоит проверить и остальные столбцы с численными значениями
* Столбец **NumberOfPictures** не содержит иных значений кроме 0, никакой полезной информации он не несет, поэтому его нужно удалить

## Предобработка данных

### Первичная предобработка данных

Прежде чем вносить какие-либо изменения, предварительно сохраним копию оригинального датасета и показатели размерности

In [4]:
original_data = data.copy()
original_data_shape = data.shape

Проверим наличие дубликатов

In [5]:
data.duplicated().sum()

4

In [6]:
# дублей мало, поэтому просто удалим их
data.drop_duplicates(inplace=True)
# проверяем, что всё удалилось
data.duplicated().sum()

0

Определим признаки, которые не важны для определения цены на автомобиль и удалим их:

1. DateCrawled — дата скачивания анкеты из базы – не влияет на стоимость авто, но перед тем, как удалить данный признак проверим крайнюю дату скачивания для определения правой границы в диапазоне года регистрации авто
2. RegistrationMonth — месяц регистрации автомобиля - месяц не даёт никакого веса при выборе автомобиля, важен только год регистрации
3. DateCreated — дата создания анкеты – не влияет на стоимость авто
4. NumberOfPictures — количество фотографий автомобиля - как мы уже определили выше, в этом столбце только нулевые значения и пользы не несут
5. PostalCode — почтовый индекс владельца анкеты (пользователя) – не влияет на стоимость авто
6. LastSeen — дата последней активности пользователя – не влияет на стоимость авто

In [7]:
data['DateCrawled'].max()

'2016-04-07 14:36:58'

Последняя дата скачивания анкеты из базы произошла в 2016 году, поэтому все данные позже 2016 года будем убирать

In [8]:
print(f'Количество столбцов до удаления: {original_data_shape[1]}')
# отбираем не информативные столбцы
columns_to_drop = ['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']
# удаляем
data.drop(columns=columns_to_drop, inplace=True)
# проверяем, что всё удалилось
display(data.columns)
print(f'Количество столбцов после удаления: {data.shape[1]}')

Количество столбцов до удаления: 16


Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'FuelType', 'Brand', 'Repaired'],
      dtype='object')

Количество столбцов после удаления: 10


Приведем названия столбцов к единому стилю

In [9]:
# приводим все названия к нижнему регистру
data.columns = data.columns.str.lower()
# редактируем столбцы, где больше одного слова
data.rename(columns = {'vehicletype': 'vehicle_type',
                       'registrationyear': 'registration_year',
                       'fueltype': 'fuel_type'}, 
            inplace=True)
# проверяем
data.columns

Index(['price', 'vehicle_type', 'registration_year', 'gearbox', 'power',
       'model', 'kilometer', 'fuel_type', 'brand', 'repaired'],
      dtype='object')

### Работа с пропусками

Теперь займемся пропусками:

In [10]:
data.isna().sum()

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64

Больше всего пропусков наблюдается у признака **repaired — была машина в ремонте или нет**. Проверим содержимое всех признаков, имеющих пропущенные значения:

In [11]:
# отбираем нужные колонки
columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired']

# выводим уникальные значения в алфавитном порядке
for column in columns:
    print(column)
    display(data[column].sort_values().unique())
    print('-' * 75)

vehicle_type


array(['bus', 'convertible', 'coupe', 'other', 'sedan', 'small', 'suv',
       'wagon', nan], dtype=object)

---------------------------------------------------------------------------
gearbox


array(['auto', 'manual', nan], dtype=object)

---------------------------------------------------------------------------
model


...

---------------------------------------------------------------------------
fuel_type


array(['cng', 'electric', 'gasoline', 'hybrid', 'lpg', 'other', 'petrol',
       nan], dtype=object)

---------------------------------------------------------------------------
repaired


array(['no', 'yes', nan], dtype=object)

---------------------------------------------------------------------------


Признаки **vehicle_type — тип автомобильного кузова**, **gearbox — тип коробки передач** и **repaired — была машина в ремонте или нет** самостоятельно лучше не заполнять модой или средним, чтобы не искажать наблюдения. 

Удалять столько строк тоже не хочется, поэтому лучшим решением будет заполнить пропуски заглушкой *not filled* – *не заполнено*

Бензин повторяется дважды под названием **gasoline** и **petrol**, это похоже на неявные дубли, поэтому стоит объединить данные под одним значением, а пропуски заполнить заглушкой

In [12]:
# заменяем все petrol на gasoline
data['fuel_type'].replace('petrol', 'gasoline', inplace=True)

# отбираем нужные колонки
columns = ['vehicle_type', 'gearbox', 'fuel_type', 'repaired']

# заполняем пропуски заглушкой
for column in columns:
    data[column].fillna('not filled', inplace=True)

А вот признак **model — модель автомобиля** рассмотрим чуть подробнее:

Модель Range Rover дублируется как **range_rover** и **rangerover**, поэтому объединим в одну категорию. 

Пропуски в этом признаке скорее всего появились из-за невозможности выбора нужной модели из выпадающего списка, но, так как это важный признак, без указания которого невозможно корректно определить стоимость авто, то лучше удалить все пропущенные значения

In [13]:
# заменяем на range_rover
data['model'].replace('rangerover', 'range_rover', inplace=True)

# заполняем пропуски
data.dropna(subset=['model'], inplace=True)

# проверяем
data.isna().sum()

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

### Работа с аномалиями / выбросами

Проверим содержимое численных признаков и подберем допустимые диапазоны значений в случае аномалий / выбросов

In [14]:
# отбираем нужные колонки
columns = ['price', 'registration_year', 'power', 'kilometer']

for column in columns:
    print(column)
    display(data[column].sort_values().unique())
    print('-' * 75)

price


array([    0,     1,     2, ..., 19998, 19999, 20000])

---------------------------------------------------------------------------
registration_year


array([1000, 1001, 1111, 1200, 1234, 1300, 1400, 1500, 1600, 1602, 1800,
       1910, 1919, 1923, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
       1935, 1936, 1937, 1938, 1941, 1942, 1943, 1945, 1947, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2290, 2500, 2900, 3000, 3700, 4000,
       4500, 5000, 5555, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800,
       8000, 8200, 8500, 9000, 9999])

---------------------------------------------------------------------------
power


array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30,    31,    32,    33,    34,    35,
          36,    37,    38,    39,    40,    41,    42,    43,    44,
          45,    46,    47,    48,    49,    50,    51,    52,    53,
          54,    55,    56,    57,    58,    59,    60,    61,    62,
          63,    64,    65,    66,    67,    68,    69,    70,    71,
          72,    73,    74,    75,    76,    77,    78,    79,    80,
          81,    82,    83,    84,    85,    86,    87,    88,    89,
          90,    91,    92,    93,    94,    95,    96,    97,    98,
          99,   100,   101,   102,   103,   104,   105,   106,   107,
         108,   109,   110,   111,   112,   113,   114,   115,   116,
         117,   118,   119,   120,   121,   122,   123,   124,   125,
         126,   127,

---------------------------------------------------------------------------
kilometer


array([  5000,  10000,  20000,  30000,  40000,  50000,  60000,  70000,
        80000,  90000, 100000, 125000, 150000])

---------------------------------------------------------------------------


1. Похоже, что поля **мощность (л. с.)** и **цена (евро)** просто не хотели заполнять и поставили 0. В таком случае стоит дать рекомендацию разработчикам ввести минимальное допустимое значение для заполнения. Некоторые значения мощности скорее всего, являются ошибочными значениями, где поставили или же не доставили одну цифру
2. В признаке **год регистрации автомобиля** много некорректных значений, здесь тоже стоит ввести нужный диапазон для заполнения
3. С **пробегом** всё в порядке

Минимальную стоимость назначим 200 евро

Год поставим с 1980 по 2016 год включительно

Мощность отберем от 30 л.с. до 1000 л.с. Например, 1000 л.с. есть у Теслы. Данные более 1000 л.с. примем за выброс, который будет только мешать в обучении модели

Посмотрим, сколько наблюдений содержится вне выбранных диапазонов:

In [15]:
def get_range(column, x, y):
    print(column.name)
    print(f'Количество наблюдений меньше {x}: {(data[column < x]).shape[0]}')
    print(f'Количество больше больше {y}: {(data[column > y]).shape[0]}')
    print()

get_range(data['price'], 200, 20000)
get_range(data['power'], 30, 1000)
get_range(data['registration_year'], 1980, 2016)

price
Количество наблюдений меньше 200: 14331
Количество больше больше 20000: 0

power
Количество наблюдений меньше 30: 34562
Количество больше больше 1000: 265

registration_year
Количество наблюдений меньше 1980: 2576
Количество больше больше 2016: 12483



Такие значения лучше удалить, так как даже у одних и тех же моделей одного года выпуска мощность может отличаться из-за разных двигателей, которые владельцы заменяли самостоятельно, а стоимость зависит от совокупности различных факторов и является целевым признаком, поэтому самостоятельно мы такие значения заполнить не сможем

In [16]:
data = data.query('(200 <= price) & (1980 <= registration_year <= 2016) & (30 <= power <= 1000)')

Проверим содержимое оставшегося признака **brand**

In [17]:
data['brand'].sort_values().unique()

array(['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen',
       'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai',
       'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda',
       'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot',
       'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart',
       'subaru', 'suzuki', 'toyota', 'trabant', 'volkswagen', 'volvo'],
      dtype=object)

Всё в порядке, можем завершать работу с аномалиями / выбросами

In [18]:
print(f'Всего удалено строк: {original_data_shape[0] - data.shape[0]}')
print(f'Это {round((original_data_shape[0] - data.shape[0]) * 100 / original_data_shape[0])}% от первоначального датасета')

Всего удалено строк: 74583
Это 21% от первоначального датасета


### Кодирование и разделение и  данных

Для обучения моделей нам будут нужны два типа данных: закодированные и нет, потому что градиентный бустинг прекрасно справляется и на оригинальных данных

In [19]:
# зафиксируем значение random_state
SEED = 12345

#### Подготовка данных для модели LightGBM

In [20]:
features = data.drop('price', axis=1)
target = data['price']

# разделим данные на обучающую и временную выборки в соотношении 70%:30%
features_train, features_sample, target_train, target_sample = train_test_split(features,
                                                                                target,
                                                                                test_size=0.3,
                                                                                random_state=SEED)

# разделим данные на валидационную и тестовую выборки в соотношении 15%:15%
features_valid, features_test, target_valid, target_test = train_test_split(features_sample,
                                                                            target_sample,
                                                                            test_size=0.5,
                                                                            random_state=SEED)


# проверим, что ничего не потеряли
data.shape[0] == features_train.shape[0] + features_valid.shape[0] + features_test.shape[0]

True

Для повышения производительности и ускорения обучения модели LightGBM преобразуем категориальные признаки из *object* в *category*

In [21]:
# отбираем категориальные признаки
categorical = features_train.dtypes[features_train.dtypes=='object'].index

# преобразовываем в тип 'category'
features_train[categorical] = (features_train[categorical].astype('category'))
features_valid[categorical] = (features_valid[categorical].astype('category'))
features_test[categorical] = (features_test[categorical].astype('category'))

# проверяем
print(features_train.dtypes)
print('-' * 50)
print(features_valid.dtypes)
print('-' * 50)
print(features_test.dtypes)

vehicle_type         category
registration_year       int64
gearbox              category
power                   int64
model                category
kilometer               int64
fuel_type            category
brand                category
repaired             category
dtype: object
--------------------------------------------------
vehicle_type         category
registration_year       int64
gearbox              category
power                   int64
model                category
kilometer               int64
fuel_type            category
brand                category
repaired             category
dtype: object
--------------------------------------------------
vehicle_type         category
registration_year       int64
gearbox              category
power                   int64
model                category
kilometer               int64
fuel_type            category
brand                category
repaired             category
dtype: object


#### Подготовка данных для линейной регрессии

In [22]:
# закодируем все категориальные признаки
features_ohe = pd.get_dummies(features, drop_first=True)
# проверим
features_ohe.head()

,registration_year,power,kilometer,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_not filled,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,...,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_not filled,repaired_yes
2,2004,163,125000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,2001,75,150000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,2008,69,90000,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
5,1995,102,150000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2004,109,150000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# разделим данные на обучающую и временную выборки в соотношении 70%:30%
features_train_ohe, features_temp_ohe, \
target_train_ohe, target_target_ohe = train_test_split(features_ohe,
                                                       target,
                                                       test_size=0.3,
                                                       random_state=SEED)

# разделим данные на валидационную и тестовую выборки в соотношении 15%:15%
features_valid_ohe, features_test_ohe, \
target_valid_ohe, target_test_ohe = train_test_split(features_temp_ohe,
                                                     target_target_ohe,
                                                     test_size=0.5,
                                                     random_state=SEED)


# проверим, что ничего не потеряли
data.shape[0] == features_train_ohe.shape[0] + features_valid_ohe.shape[0] + features_test_ohe.shape[0]

True

#### Подготовка данных для случайного леса

In [24]:
# сохраняем копию
features_oe = features.copy()
# инициализируем OrdinalEncoder
encoder = OrdinalEncoder()
# отбираем категориальные признаки
cat_columns = features_oe.columns[features_oe.dtypes == 'object'].tolist()
# кодируем
features_oe[cat_columns] = encoder.fit_transform(features_oe[cat_columns])
# проверим
features_oe.head()

,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
2,7.0,2004,0.0,163,117.0,125000,2.0,14.0,1.0
3,6.0,2001,1.0,75,116.0,150000,2.0,37.0,0.0
4,6.0,2008,1.0,69,101.0,90000,2.0,31.0,0.0
5,5.0,1995,1.0,102,11.0,150000,2.0,2.0,2.0
6,1.0,2004,1.0,109,8.0,150000,2.0,25.0,0.0


In [25]:
# разделим данные на обучающую и временную выборки в соотношении 70%:30%
features_train_oe, features_temp_oe, \
target_train_oe, target_temp_oe = train_test_split(features_oe,
                                                   target,
                                                   test_size=0.3,
                                                   random_state=SEED)

# разделим данные на валидационную и тестовую выборки в соотношении 15%:15%
features_valid_oe, features_test_oe, \
target_valid_oe, target_test_oe = train_test_split(features_temp_oe, 
                                                    target_temp_oe,
                                                    test_size=0.5,
                                                    random_state=SEED)

# проверим, что ничего не потеряли
data.shape[0] == features_train_oe.shape[0] + features_valid_oe.shape[0] + features_test_oe.shape[0]

True

## Обучение моделей

### Линейная регрессия

In [26]:
%%time

# обучаем и замеряем время обучения
start = time.time()
model_lr = LinearRegression()
model_lr.fit(features_train_ohe, target_train_ohe)
train_time_lr = round(time.time() - start, 2)

print(f'Время обучения: {train_time_lr} сек.')
print()

Время обучения: 22.34 сек.

CPU times: user 13.8 s, sys: 8.56 s, total: 22.4 s
Wall time: 22.3 s


In [27]:
%%time

# замеряем время предсказания
start = time.time()
predictions_lr = model_lr.predict(features_valid_ohe)
pred_time_lr = round(time.time() - start, 2)

# вычисляем RMSE
rmse_lr = mean_squared_error(target_valid_ohe, predictions_lr)**0.5

print(f'Время предсказания: {pred_time_lr} сек.')
print(f'RMSE: {rmse_lr}')
print()

Время предсказания: 0.22 сек.
RMSE: 2401.660031951627

CPU times: user 91.9 ms, sys: 74.7 ms, total: 167 ms
Wall time: 221 ms


### Случайный лес

In [28]:
%%time

parameters = {
    'max_depth': [2, 6],
    'n_estimators': range(10, 100, 10)
}

# подбираем лучшие параметры
model_rf = RandomizedSearchCV(RandomForestRegressor(random_state=SEED),
                              parameters,
                              random_state=SEED,
                              cv=2, 
                              scoring='neg_root_mean_squared_error',
                              n_jobs=-1)

model_rf.fit(features_train_oe, target_train_oe)

print(model_rf.best_params_)
print()

{'n_estimators': 20, 'max_depth': 6}

CPU times: user 52.2 s, sys: 112 ms, total: 52.3 s
Wall time: 52.6 s


In [29]:
%%time

# обучаем и замеряем время обучения
start = time.time()
model_rf = RandomForestRegressor(random_state=SEED, max_depth=6)
model_rf.fit(features_train_oe, target_train_oe)
train_time_rf = round(time.time() - start, 2)

print(f'Время обучения: {train_time_rf} сек.')
print()

Время обучения: 14.94 сек.

CPU times: user 14.9 s, sys: 23.8 ms, total: 14.9 s
Wall time: 14.9 s


In [30]:
%%time

# замеряем время предсказания
start = time.time()
predictions_rf = model_rf.predict(features_valid_oe)
pred_time_rf = round(time.time() - start, 2)

# вычисляем RMSE
rmse_rf = mean_squared_error(target_valid_oe, predictions_rf)**0.5

print(f'Время предсказания: {pred_time_rf} сек.')
print(f'RMSE: {rmse_rf}')
print()

Время предсказания: 0.19 сек.
RMSE: 2158.4947633445527

CPU times: user 185 ms, sys: 0 ns, total: 185 ms
Wall time: 192 ms


### LightGBM

In [31]:
%%time

parameters = {'max_depth': [2, 6]}

# подбираем параметры
model_lgbm = RandomizedSearchCV(LGBMRegressor(random_state=SEED),
                                parameters,
                                random_state=SEED,
                                cv=2,
                                scoring='neg_root_mean_squared_error',
                                n_jobs=-1)

model_lgbm.fit(features_train, target_train)

print(model_lgbm.best_params_)
print()

{'max_depth': 6}

CPU times: user 27.3 s, sys: 57.7 ms, total: 27.3 s
Wall time: 27.5 s


In [32]:
%%time

# обучаем и замеряем время обучения
start = time.time()
model_lgbm = LGBMRegressor(random_state=SEED, max_depth=6, learning_rate=1.0)
model_lgbm.fit(features_train, target_train)
train_time_lgbm = round(time.time() - start, 2)

print(f'Время обучения: {train_time_lgbm} сек.')
print()

Время обучения: 3.6 сек.

CPU times: user 3.6 s, sys: 13.2 ms, total: 3.61 s
Wall time: 3.6 s


In [33]:
%%time

# замеряем время предсказания
start = time.time()
predictions_lgbm = model_lgbm.predict(features_valid)
pred_time_lgbm = round(time.time() - start, 2)

# вычисляем RMSE
rmse_lgbm = mean_squared_error(target_valid, predictions_lgbm)**0.5

print(f'Время предсказания: {pred_time_lgbm} сек.')
print(f'RMSE: {rmse_lgbm}')
print()

Время предсказания: 0.31 сек.
RMSE: 1589.2580926011983

CPU times: user 366 ms, sys: 0 ns, total: 366 ms
Wall time: 392 ms


### Константная модель

In [34]:
%%time

# обучаем и замеряем время обучения
start = time.time()
constant_model = DummyRegressor(strategy='mean')
constant_model.fit(features_train, target_train)
train_time_cm = round(time.time() - start, 3)

print(f'Время обучения: {train_time_cm} сек.')
print()

Время обучения: 0.001 сек.

CPU times: user 2.6 ms, sys: 0 ns, total: 2.6 ms
Wall time: 1.47 ms


In [35]:
%%time

# замеряем время предсказания
start = time.time()
predictions_cm = constant_model.predict(features_valid)
pred_time_cm = round(time.time() - start, 3)

# вычисляем RMSE
rmse_cm = mean_squared_error(target_valid, predictions_cm)**0.5

print(f'Время предсказания: {pred_time_cm} сек.')
print(f'RMSE: {rmse_cm}')
print()

Время предсказания: 0.0 сек.
RMSE: 4614.730863581237

CPU times: user 1.41 ms, sys: 0 ns, total: 1.41 ms
Wall time: 1.07 ms


## Анализ моделей

In [36]:
compare = pd.DataFrame({
    'Модель': ['Линейная регрессия', 'Случайный лес', 'LightGBM', 'Константная модель'],
    'Значение RMSE': [rmse_lr, rmse_rf, rmse_lgbm, rmse_cm],
    'Время обучения': [train_time_lr, train_time_rf, train_time_lgbm, train_time_cm],
    'Время предсказания': [pred_time_lr, pred_time_rf, pred_time_lgbm, pred_time_cm]
})

compare

,Модель,Значение RMSE,Время обучения,Время предсказания
0,Линейная регрессия,2401.660032,22.340,0.22
1,Случайный лес,2158.494763,14.940,0.19
2,LightGBM,1589.258093,3.600,0.31
3,Константная модель,4614.730864,0.001,0.00


Лучшую скорость обучения и значение RMSE показала модель градиентного бустинга LightGBM. Во время предсказания немного подкачала по скорости в сравнении со всеми моделями, но учитывая такую высокую скорость обучения на большом объеме данных при двух гиперпараметрах в то время, когда у случайного леса только один – на это можно закрыть глаза и считать данную модель лучшей

Также наблюдается высокая скорость обучения и предсказания константной модели, но низкое качество перебивает такие плюсы – RMSE составляет более 4к, а это очень плохой показатель

Проверим теперь LightGBM на тестовой выборке

In [37]:
%%time

# замеряем время предсказания на тестовой выборке
start = time.time()
predictions_lgbm = model_lgbm.predict(features_test)
pred_time_lgbm = round(time.time() - start, 2)

# вычисляем RMSE
rmse_lgbm = mean_squared_error(target_test, predictions_lgbm)**0.5

print(f'Время предсказания: {pred_time_lgbm} сек.')
print(f'RMSE: {rmse_lgbm}')
print()

Время предсказания: 0.36 сек.
RMSE: 1604.7741959498553

CPU times: user 353 ms, sys: 0 ns, total: 353 ms
Wall time: 362 ms


### Вывод

<div style="border:solid silver 2px; padding: 30px">
    
LightGBM и на тестовой выборке не подкачала и показывает адекватные результаты

##  Общий вывод

На основании исследования, сервису по продаже автомобилей с пробегом «Не бит, не крашен» стоит порекомендовать модель градиентного бустинга LightGBM с параметром max_depth=6

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей